# Working with CIFTI files

## Types of cifti-files
Cifti files can be pretty handy to store imaging data in a flexible way. They can store data in a 2D matrix, with the two axis defining the type of data you are looking at. This allows for a lot of flexibility in how data is stored and displayed.

There are 5 different kinds of axis:
- BrainModelAxis: each row/column is a voxel or vertex - multiple brainModelAxis can be combined into a single axis
- ParcelsAxis: each row/column is a group of voxels and/or vertices
- ScalarAxis: each row/column has a unique name (with optional meta-data)
- LabelAxis: each row/column has a unique name and label table (with optional meta-data)
- SeriesAxis: each row/column is a timepoint, which increases monotonically

The two different axes define a set of different type of CIFTI files - each with a different extension. The column axis usually is a BrainModel or Parcels axis - this is the space that is being displayed. The rows is the type of data that is displayed in that space.

| File extension | Description | Row Axis | Column Axis |
|---------|---------|---------|---------|
| .dscalar.nii | Dense scalar data | Scalar | BrainModel |
| .pscalar.nii | Parcel scalar data | Scalar | Parcel |
| .dtseries.nii | Dense Timeseries | Series | BrainModel |
| .ptseries.nii | Parcel Timeseries | Series | Parcel |
| .dlabel.nii | Dense label data | Label | BrainModel |
| .plabel.nii | Parcel label data | Label | parcel |
| .dconn.nii | Dense connectivity | BrainModel | BrainModel |
| .pconn.nii | ROI-to-ROI connectivity | Parcel | Parcel |
| .dpconn.nii | Dense-to-ROI connectivity | Parcel | BrainModel |
| .pdconn.nii | ROI-to-dense connectivity | BrainModel | Parcel |

In [2]:
import sys
from os import getcwd
from os.path import join, exists, abspath
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colormaps, cm, colors
import scipy
import seaborn as sns

from tqdm import tqdm

import nibabel as nb
import nitools as nt

In [3]:
dir_current = getcwd()

list_ = []
for ii, dname in enumerate(dir_current.split('\\')[:-3]):
    if ii==0:
        tmp = dname+'/'
    else:
        tmp = dname
    list_.append(tmp)
dir_git = join(*list_)
dir_git

'D:/mobaxterm\\sungbeenpark\\github'

In [4]:
path_module = abspath(join(dir_git,'Functional_Fusion'))
if exists(path_module):
    print('added the path, %s'%path_module)
    if path_module not in sys.path:
        sys.path.append(path_module)

import Functional_Fusion.atlas_map as am
import Functional_Fusion.dataset as ds
import Functional_Fusion.util as ut
import Functional_Fusion.import_data as id
import Functional_Fusion.array_convert as ac

added the path, D:\mobaxterm\sungbeenpark\github\Functional_Fusion


In [5]:
# path_module = abspath(join(dir_git,'SUITPy'))
# if exists(path_module):
#     print('added the path, %s'%path_module)
#     if path_module not in sys.path:
#         sys.path.append(path_module)

# import SUITPy as suit

In [6]:
path_module = abspath(dir_git)
if exists(path_module):
    print('added the path, %s'%path_module)
    if path_module not in sys.path:
        sys.path.append(path_module)

import surfAnalysisPy as surf

added the path, D:\mobaxterm\sungbeenpark\github


In [7]:
dir_work = join('D:/milli/diedrichsenlab/SeqSpatialSupp_fMRI')
dir_surf = join(dir_work,'surfaceWB')
dir_preproc = join(dir_work,'imaging_data')

---

## Atlas Spaces

Each Atlas is defined in a specific space. The atlas space defines how data is being transformed between Native space and the atlas, or between different atlases. `Function_Fusion` follows the convention of atlas space naming used in [Templateflow.org](https://www.templateflow.org/).

The following spaces are defined so far:
- `MNI152NLin6Asym`: MNI152NLin6Asym space, standard MNI space template in FSL
- `MNI152NLin2009cSym`: Symmetric version of the MNI152NLin2009c space
- `MNI152NLin2009cAsym`: Asymmetric version of the MNI152NLin2009c space
- `SUIT`: SUIT space, a cerebellar template space
- `MNI152NLin2009cSymC`: Symmetric Cerebellum-only atlas (aligned to )
- `fs32k`: Symmetric surface-based space for left and right hemisphere
- `fsaverage6`: Freesurfer fsaverage6 space - needs to be still implemented

Templates in different resolutions and the mask images that define the Atlases (see below) can be found in the directory `Functional_Fusion/Atlases/tpl-<space>`.

## Standard Atlases

Atlases are defined by a specific space and a mask image that defines the resolution and the voxels / vertices that are included in the atlas. Standard atlases are indicated by short strings that indicate the Atlas. The following Atlases are defined so far: * `SUIT3`: Cerebellum in SUIT space (3mm voxels) * `SUIT2`: Cerebellum in SUIT space (2mm voxels) * `SUIT1`: Cerebellum in SUIT space (1mm voxels) * `MNISymC3`: Cerebellum in MNI152NLin2009cSym space (3mm voxels) * `MNISymC2`: Cerebellum in MNI152NLin2009cSym space (2mm voxels) * `fs32k`: Left and Right hemisphere, using symmetric Medial wall mask * `fs32kAsym`: Left and Right hemisphere, using asymmetric medial wall mask from HCP. * `MNIAsymBg1`: Basal Ganglia in MNI152NLin6Asym space (1mm voxels) * `MNIAsymBg2`: Basal Ganglia in MNI152NLin6Asym space (2mm voxels)

All these atlases are defined in `Functional_Fusion/Atlases/atlas_description.json`. In this file, each atlas is defined by a dictionary with the following fields: * `type`: Class-name for the atlas object (“AtlasVolumetric”, “AtlasSurface”, …) * `dir`: Directory where the atlas files are stored: Functional_Fusion/Atlases/<dir>) * `mask`: Name(s) of mask niftis/gifti files that define the atlas space. * space: Space in which coordinates are defined (e.g. “MNI152NLin6AsymC”) * `structure`: cifti-structure name - needs to be a valid, predefined cifti structure (e.g. “cerebellum”)

Optional fields:
- `res`: resolution (not used yet) -voxel locations are defined by mask image
- `normspace`: Norm space for suit.vol2surf -

## Getting Atlases

In [8]:
fs32L = am.get_atlas('fs32k_L')

In [14]:
subj = 'S01'
S_id = subj.replace('R','S')

In [13]:
glm = 2
dir_glm = join(dir_work,'glm_%d'%glm)
mask = join(dir_glm,'%s/mask.nii'%subj)

In [ ]:
# wsurf = data_dir + '/surfaceWB/sub-01/sub-01.L.white.32k.surf.gii'
# psurf = data_dir + '/surfaceWB/sub-01/sub-01.L.pial.32k.surf.gii'
# mask = data_dir +  '/glm/glm_1/sub-01/mask.nii'

In [19]:
wsurf = join(dir_surf,'%s/%s.L.white.32k.surf.gii'%(S_id,S_id))
psurf = join(dir_surf,'%s/%s.L.pial.32k.surf.gii'%(S_id,S_id))

In [7]:
A = nb.load('fs_LR.32k.L.ventralmotor.func.gii')
data = A.darrays[0].data
# Find the vertices in the ROI 
vertices = np.where(data != 0)[0]
filenames = ['con_ba','con_da']
dataf = [f'{data_dir}/glm/glm_1/sub-01/{f}.nii' for f in filenames]
atlas_map = am.AtlasMapSurf(vertices,wsurf,psurf,mask)
atlas_map.build()

atlas_map.vox_list

FileNotFoundError: No such file or no access: 'fs_LR.32k.L.ventralmotor.func.gii'

---

## Extract Data